# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [122]:
import os
import pandas as pd
import pathlib


In [123]:
pub_files = os.listdir("../_publications")

tsv_content = []
for pub_dir in pub_files:
    pub_file = pathlib.Path(pub_dir)
    content = open("../_publications" / pub_file, "r").read()
    dict_content = ([item for item in content.split("---")[1].split("\n") if len(item) > 0 and item[0] != "#"])
    dict_content = dict([(item.split(": ")[0], item.split(": ")[1]) for item in dict_content])
    dict_content["url_slug"] = pub_file.stem
    if dict_content.get("setup", "generated") == "primary":
        tsv_content.append(dict_content)

tsv_content = pd.DataFrame(tsv_content)
keys = ["pub_date","title","venue","excerpt","citation","url_slug","paper_url","slides_url"]
tsv_df = pd.DataFrame(columns=keys)
tsv_df = tsv_df.append(tsv_content).fillna('')
tsv_df[keys].to_csv("publications.tsv", sep="\t", index=False)
tsv_df[keys]

/var/folders/bn/rzkm0nsd21163qwht8y70_8r0000gn/T/ipykernel_74398/1732166078.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tsv_df = tsv_df.append(tsv_content).fillna('')


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2023-05-01,Conformal Prediction Sets for Graph Neural Net...,'ICML',We show that conformal prediction is applicabl...,"<i>Zargarbashi, Soroush H., Simone Antonelli, ...",2023-05-01-daps,https://proceedings.mlr.press/v202/h-zargarbas...,


In [124]:
pub_file.stem

'2023-05-01-daps'

In [125]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2023-05-01	Conformal Prediction Sets for Graph Neural Networks	'ICML'	We show that conformal prediction is applicable (valid) to graph node classification, and we further show that diffusion of conformity scores results in smaller (more efficient) prediction sets.	"<i>Zargarbashi, Soroush H., Simone Antonelli, and Aleksandar Bojchevski. ""Conformal prediction sets for graph neural networks."" International Conference on Machine Learning. PMLR, 2023.</i>"	2023-05-01-daps	https://proceedings.mlr.press/v202/h-zargarbashi23a.html	


## Import pandas

We are using the very handy pandas library for dataframes.

In [126]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [127]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2023-05-01,Conformal Prediction Sets for Graph Neural Net...,'ICML',We show that conformal prediction is applicabl...,"<i>Zargarbashi, Soroush H., Simone Antonelli, ...",2023-05-01-daps,https://proceedings.mlr.press/v202/h-zargarbas...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [128]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [129]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [130]:
!ls ../_publications/

2023-05-01-2023-05-01-daps.md 2023-05-01-daps.md


In [131]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
